In [ ]:
!pip install tensorflow pandas scikit-learn nltk

In [ ]:
!pip install transformers torch


In [ ]:
# 1. Import Libraries
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

# 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 3. Akses direktori dataset
base_path = '/content/drive/MyDrive/DATASET TEXT ANALYSIS/DATASET PRE PROCESSING/bbc'
categories = ['business', 'entertainment', 'politics', 'sport', 'tech']

# 4. Membaca semua file dari setiap kategori dan menyimpan label
docs = []
labels = []

for category in categories:
    category_path = os.path.join(base_path, category)
    for filename in os.listdir(category_path):
        if filename.endswith('.txt'):  # Pastikan hanya membaca file .txt
            file_path = os.path.join(category_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                docs.append(f.read())
                labels.append(category)

# 5. Memetakan label ke angka
label_to_id = {label: id for id, label in enumerate(set(labels))}
id_to_label = {id: label for label, id in label_to_id.items()}
encoded_labels = [label_to_id[label] for label in labels]

# 6. Memisahkan dataset menjadi data latih dan uji
train_texts, test_texts, train_labels, test_labels = train_test_split(
    docs,
    encoded_labels,
    test_size=0.2,
    random_state=42
)

# 7. Inisialisasi tokenizer dari DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# 8. Tokenisasi data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# 9. Mengonversi ke TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(16)

# 10. Memuat model DistilBERT untuk klasifikasi
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_to_id))

# 11. Menyiapkan optimizer dan loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.metrics.SparseCategoricalAccuracy('accuracy')

# 12. Kompilasi model
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# 13. Melatih model
model.fit(train_dataset, epochs=3)

# 14. Evaluasi
eval_result = model.evaluate(test_dataset)

# Output hasil evaluasi
print(f"Hasil evaluasi: {eval_result}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistil

Epoch 1/3
112/112 [==============================] - 126s 923ms/step - loss: 0.4004 - accuracy: 0.9107
Epoch 2/3
112/112 [==============================] - 105s 942ms/step - loss: 0.0914 - accuracy: 0.9792
Epoch 3/3
28/28 [==============================] - 11s 326ms/step - loss: 0.0395 - accuracy: 0.9865
Hasil evaluasi: [0.03953028470277786, 0.9865168333053589]


In [ ]:
# 1. Import Libraries
import numpy as np
from sklearn.metrics import f1_score  # Import f1_score
import tensorflow as tf

# 14. Evaluasi
eval_result = model.evaluate(test_dataset)

# 15. Memprediksi label untuk data pengujian
predictions = model.predict(test_dataset)
predicted_labels = np.argmax(predictions.logits, axis=1)  # Ambil label dengan probabilitas tertinggi

# 16. Hitung F1 Score
f1 = f1_score(test_labels, predicted_labels, average='weighted')

# Output F1 Score
print(f"F1 Score: {f1}")


28/28 [==============================] - 11s 341ms/step
F1 Score: 0.9707059283799496
